# CLIMATE DATA CHALLENGE

## Understanding Drivers of Uncerainty in Future Loss Projections

### Exposure scenario, sensitivity to:

- Popultation increases
- locality
- building type
- cost

In [1]:
# Load libraries

import sys

import numpy as np

import pandas as pd

import ascend
from ascend import shape

import datetime

import iris

from matplotlib import pyplot as plt
import matplotlib.cm as cm

#import warnings
#warnings.filterwarnings('ignore')

In [2]:
# Define paths to data

datadir = "/net/home/h04/jmendes/ClimateDataChallenge/Met-Office-Climate-Data-Challenge-May-2022/data"
ssp_data = "/data/users/ldawkins/UKCR/DataForPaper/UKSSPs"

In [8]:
# Functions

def read_file(datadir, country="all"):
    '''
    Read files from input directory.
    
    Args:
        datadir (str): Path to input data dir
        country (list (str)): Country to be selected.
                              Default is to load all countries.
    
    Returns:
        Pandas dataframe of input data with 15-17 columns.
    '''

    # define filename
    file = f"{datadir}/UKMO_HACKATHON_EXPOSURE.csv"

#    # define names of columns
#    columns = [
#        "LocationSID",
#        "LocationName",
#        "BuildingsReplacement",
#        "ContentsReplacement",
#        "BusinessInteruptionReplacement",
#        "CurrencyCode",
#        "Territory",
#        "CountryName",
#        "PostalCode",
#        "PostalName",
#        "Latitude",
#        "Longitude",
#        "Stories",
#        "Occupancy",
#        "Construction"
#    ]
    
    # read file as dataframe and return
#    return pd.read_csv(file, header = 1, names = columns)
    # Make a one column dataframe with each row being a line in the CSV file
    # original file is latin1 encoded
    df = pd.read_fwf(f"{datadir}/UKMO_HACKATHON_EXPOSURE.csv", header=None, encoding='latin1')
    # split each row on commas and expand the dataframe
    if country == "all":
        return df[0].str.split(',', expand=True)
    else:
        dfmod = df[0].str.split(',', expand=True)
        return dfmod.loc[dfmod.iloc[:,7]==country]

#TODO: add multiple countries, isin?   


In [4]:
def convert_time(cube):
    '''
    Convert cube's time coordinate onto decade.
    
    Args:
        cube(iris cube): Input SSP cube.
                         time units are 'hours since 1970-01-01 00:00:00'
                         in Gregorian calendar.
    
    Returns:
        List of decades (datetime year format)
    '''


    decades = []
    for t in cube.coord("time"):
        decades.append(datetime.datetime.fromtimestamp(int(t.points)*3600).year)
    
    return decades

In [9]:
data = read_file(datadir, country="United Kingdom")

In [10]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
430,272305,48_United Kingdom,1730354082.412524,865177041.20626199,865177041.20626199,EUR,GBR,United Kingdom,BT29 4RT,BT29 4RT,54.609100,-6.335300,4,General commercial,Reinforced concrete MRF w/ URM,None,None,None
7798,286329,78_United Kingdom,8001549283.6886797,4000774641.8443398,4000774641.8443398,EUR,GBR,United Kingdom,FK13 6LY,FK13 6LY,56.133700,-3.729800,1,General commercial,Reinforced concrete MRF w/ URM,None,None,None
7978,286506,81_United Kingdom,2528671562.157321,1264335781.07866,1264335781.07866,EUR,GBR,United Kingdom,FK13 6LY,FK13 6LY,56.133700,-3.729800,4,General commercial,Reinforced concrete MRF w/ URM,None,None,None
9421,305569,46_United Kingdom,154890746.21993601,77445373.109968007,77445373.109968007,EUR,GBR,United Kingdom,BT29 4RT,BT29 4RT,54.609100,-6.335300,6,General commercial,Reinforced concrete MRF w/ URM,None,None,None
9485,305725,50_United Kingdom,51630248.739978999,25815124.369989,25815124.369989,EUR,GBR,United Kingdom,BT29 4RT,BT29 4RT,54.609100,-6.335300,6,General commercial,Reinforced concrete MRF w/ URM,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2454461,315833,129_United Kingdom,7541208975.8136396,3770604487.9068198,3770604487.9068198,EUR,GBR,United Kingdom,LD4 4ES,LD4 4ES,52.090600,-3.592200,1,General commercial,Unreinforced masonry bearing frame,None,None,None
2454587,315994,134_United Kingdom,1662349860.763505,831174930.38175201,831174930.38175201,EUR,GBR,United Kingdom,LD4 4ES,LD4 4ES,52.090600,-3.592200,4,General commercial,Unreinforced masonry bearing frame,None,None,None
2455727,332557,58_United Kingdom,984707575.053859,492353787.52692902,492353787.52692902,EUR,GBR,United Kingdom,BT29 4RT,BT29 4RT,54.609100,-6.335300,4,General commercial,Unreinforced masonry bearing frame,None,None,None
2458209,137129,18_United Kingdom,33292404826.163597,16646202413.081799,16646202413.081799,EUR,GBR,United Kingdom,CV23 0WE,CV23 0WE,52.401500,-1.242500,2,General commercial,Unreinforced masonry bearing frame,None,None,None


In [ ]:
uk = dfmod.loc[dfmod.iloc[:,7]=="United Kingdom"]

In [ ]:
shp = shape.load_shp(f"{ssp_data}/Control_scenario_UK.shp")

In [ ]:
shp

In [ ]:
shape.show(shp, bounds=[-20, 40, 35, 72], scale="50m")

In [12]:
pop1 = iris.load_cube(f"{ssp_data}/population_SSP1_12km.nc")

In [13]:
pop1_decades = convert_time(pop1)

In [14]:
pop1_decades

[2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]

In [ ]:
# average annual loss by region (what loss is going to be in each year)
# or in EP basis - CAP? model: exposure component - take climate data; 

# number of storeys, construction ...
#we're only going to modify cost fields for the UK
# average annual loss across UK, change in EP curve

In [ ]:
# MAIN
def main():
    '''
    Loop through changes.
    Plot.
    '''


In [ ]:
if __name__ == '__main__':
    main()